In [ ]:
from sparseDPD import Volterra
from sparseDPD import Dataset
from sparseDPD import Datapath
from sparseDPD import DataManager
from sparseDPD import NeuralNetwork
from sparseDPD import DeltaGRUNetwork


In [ ]:
simpleDataManager = DataManager(filepath='PA_IO.mat', num_training_points=10000, num_validaiton_points=10000, num_test_points=2000)

# Setup a volterra model and a volterra inverse
volterra_forward_model = Volterra(num_nl_orders=5, num_memory_levels=3, dataset=simpleDataManager.training_dataset)
volterra_invserse_model = Volterra(num_nl_orders=5, num_memory_levels=3, dataset=simpleDataManager.training_dataset)

print(f" Volterra NMSE: {volterra_forward_model.calculate_volterra_nmse(simpleDataManager.test_dataset)} dB")
training_dataset = simpleDataManager.training_dataset

# Traing small Inverse Model 
simple_nn_inv = NeuralNetwork(num_memory_levels=5, model_type='PNTDNN', forward_model=False)
train_losses, valid_losses, best_epoch = simple_nn_inv.get_best_model(num_epochs=100, training_dataset=training_dataset, validation_dataset=simpleDataManager.validation_dataset, learning_rate=1e-3)

# Create datapath with volterra forward model and nn inverse model
datapath = Datapath(forward_model=volterra_forward_model, inverse_model=simple_nn_inv)
# Process test data through datapath
test_dataset = datapath.process(simpleDataManager.test_dataset.input_data)
# Calculate NMSE
nmse = test_dataset.calculate_nmse()
print(f"Test NMSE: {nmse:.2f} dB")
# Plot input vs output magnitude
Datapath.plot_signals(test_dataset)

In [ ]:
# Test NN forward performance
forward_nn = NeuralNetwork(num_memory_levels=5, model_type='PNTDNN_Deep', forward_model=True)
train_losses_fwd, valid_losses_fwd, best_epoch_fwd = forward_nn.get_best_model(num_epochs=200, training_dataset=simpleDataManager.training_dataset, validation_dataset=simpleDataManager.validation_dataset, learning_rate=1e-2)
# Print NMSE for forward model
fwd_nmse = forward_nn.calculate_forward_nmse(simpleDataManager.test_dataset)
print(f"NN Forward Model NMSE: {fwd_nmse:.2f} dB")

Using cpu device
Epoch  10/200  Loss=6.5673e-01  Valid Loss=5.9654e-01  LR=1.00e-02


In [ ]:
# Use OpenDPD dataset, store the folder containing the csv files
openDPD_folder = 'OpenDPD_datasets/DPA_200MHz'

openDPD_dataManager = DataManager(
    num_test_points=2000,
    num_training_points=10000,
    num_validaiton_points=10000,
    openDPD_test_input_file=f'{openDPD_folder}/test_input.csv',
    openDPD_test_output_file=f'{openDPD_folder}/test_output.csv',
    openDPD_training_input_file=f'{openDPD_folder}/train_input.csv',
    openDPD_training_output_file=f'{openDPD_folder}/train_output.csv',
    openDPD_validation_input_file=f'{openDPD_folder}/val_input.csv',
    openDPD_validation_output_file=f'{openDPD_folder}/val_output.csv'
)

# Setup a volterra model and a volterra inverse
volterra_forward_model_openDPD = Volterra(num_nl_orders=5, num_memory_levels=3, dataset=openDPD_dataManager.training_dataset)
volterra_invserse_model_openDPD = Volterra(num_nl_orders=5, num_memory_levels=3, dataset=openDPD_dataManager.training_dataset)

print(f" OpenDPD Volterra NMSE: {volterra_forward_model_openDPD.calculate_volterra_nmse(openDPD_dataManager.test_dataset)} dB")

 OpenDPD Volterra NMSE: -28.33605172879924 dB


In [9]:
# Train forward NN on OpenDPD data
open_forward_nn = NeuralNetwork(num_memory_levels=5, model_type='PNTDNN_Deep', forward_model=True)
train_losses_open, valid_losses_open, best_epoch_open = open_forward_nn.get_best_model(num_epochs=100, training_dataset=openDPD_dataManager.training_dataset, validation_dataset=openDPD_dataManager.validation_dataset, learning_rate=1e-3)

Using cpu device
Epoch  10/100  Loss=8.1862e+00  Valid Loss=4.5164e+00  LR=1.00e-03
Epoch  20/100  Loss=7.7528e+00  Valid Loss=3.5154e+00  LR=1.00e-03
Epoch  30/100  Loss=7.6440e+00  Valid Loss=3.7073e+00  LR=1.00e-03
Epoch  40/100  Loss=6.5718e+00  Valid Loss=3.2485e+00  LR=5.00e-04
Epoch  50/100  Loss=6.6210e+00  Valid Loss=4.2760e+00  LR=5.00e-04
Epoch  60/100  Loss=6.5333e+00  Valid Loss=3.1941e+00  LR=5.00e-04
Epoch  70/100  Loss=6.1482e+00  Valid Loss=3.1364e+00  LR=2.50e-04
Epoch  80/100  Loss=5.9891e+00  Valid Loss=3.1693e+00  LR=1.25e-04
Epoch  90/100  Loss=5.9624e+00  Valid Loss=3.4629e+00  LR=1.25e-04
Epoch 100/100  Loss=5.8569e+00  Valid Loss=3.0687e+00  LR=6.25e-05

Best model from epoch 88 with validation loss: 3.0681e+00


In [10]:
#print the nmse of the forward nn 
fwd_nmse_openDPD = delta_fwd.calculate_forward_nmse(openDPD_dataManager.test_dataset)
print(f"OpenDPD NN Forward Model NMSE: {fwd_nmse_openDPD:.2f} dB")

# print the number of points in openDPD dataset
print(f"OpenDPD {len(openDPD_dataManager.input_data)} points loaded.")

OpenDPD NN Forward Model NMSE: -22.90 dB
OpenDPD 38400 points loaded.
